In [1]:
import os
import pandas as pd
import numpy as np
from uuid import uuid4
from bson import Binary
import pickle

In [2]:
# NBVAL_IGNORE_OUTPUT
n,m = 5000, 50
frame = pd.DataFrame(data=np.random.randn(n, m), columns=[str(uuid4()) for _ in range(0, m)])
frame

,c9b69ee4-4ab4-40e9-a477-1c4da352c4a3,2096984e-f2d9-4c8c-b9c2-b24e36acb997,3df3298c-b9d4-419d-9254-2173e5b7025d,0aeac789-7080-4a48-b3fc-3ae2ab966ab7,037ab78c-986e-4cc5-a2bd-fcde9d9dc6db,2b219980-602a-4faa-94d0-482cd34c96b3,a6eff9db-7c4b-444c-8d8f-8db8760bdd8c,cb4edc38-dca6-4bae-81bc-f7cb15ba30d8,415d8b32-91ca-49a1-8590-c2209ff2b01c,40e78704-8a40-4ff0-9f78-b0b780d256d1,...,8a7de75a-8607-49a1-a964-83618f68fe48,d6678482-a498-4201-8fda-546e3bad880d,facd5c75-71b6-4040-a22c-efb728a88f90,70ded521-9c85-42d0-b83d-2eeeaa5d3617,9881e5c4-995d-4542-b8e0-22876399d101,02a296f7-2404-4abd-b4c0-537b539b1e6e,ab0ac558-488c-40ca-8560-aa113f599be2,298bc8fd-7791-4d7b-90a9-977c2779a030,1bb73fc8-7775-48c3-a208-a24f29458017,3c8ddff0-ee6c-4a14-89c8-0dab7b0654ae
0,-0.463464,-1.001126,-0.203145,0.621659,-2.208092,-0.168696,-0.223169,1.070284,-1.636822,1.106515,...,1.072453,-0.297574,-1.522999,1.324187,-1.121323,1.044061,-0.691973,0.624476,0.180432,-0.435217
1,1.635896,-0.932067,0.224132,-0.332059,0.390441,-0.588322,-0.131080,-2.026877,0.822169,0.705287,...,0.935194,1.101541,-0.247370,-0.091505,-1.124738,1.219574,0.030081,-0.051380,0.512421,0.334188
2,-0.222456,0.495723,-0.447721,-1.068711,0.559677,-2.364548,2.358682,-1.221849,-0.262886,-0.457432,...,0.773010,1.024720,0.087750,-1.157776,-0.384998,2.700599,-0.251678,-0.440501,1.364179,0.978625
3,0.150647,0.198249,-0.039927,0.531216,-1.614390,0.351248,0.766579,0.587884,-1.205876,-1.203238,...,-0.058783,-2.056866,0.177481,0.399555,1.068448,0.003033,-0.220717,1.217884,0.170159,1.972958
4,-2.133019,0.092633,-0.913353,-1.536125,-0.376449,2.376477,-0.425229,-0.025551,-1.158880,0.520701,...,1.063670,1.029050,0.176923,-0.222849,-1.048064,0.483655,0.072907,-0.390746,-0.535385,0.434312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.129620,-0.159527,0.033364,0.457540,0.271394,-1.335157,-1.780176,-0.323366,0.394001,2.140053,...,0.402198,-0.220813,-0.259354,0.068554,0.792502,-1.742427,0.662225,-0.164646,-0.641332,0.154955
4996,-1.093936,0.258119,-0.910916,-1.697625,-1.097885,0.000657,-0.215331,-0.413760,-0.275009,1.023765,...,-0.383740,1.526043,-0.746579,0.750673,1.381006,-1.253256,0.904038,-1.244581,-0.572822,-0.121798
4997,-1.353825,1.104814,-1.815449,1.832284,-0.219539,-0.957594,-1.486728,-0.304389,0.760631,-0.824442,...,-2.532206,0.774882,1.254331,-0.044619,-1.139742,-0.637408,0.452459,-0.133990,-0.144986,0.303030
4998,0.259666,-0.723719,-0.785082,-0.465021,-0.729696,-0.650819,-0.883028,0.259015,0.597361,-1.614041,...,-0.568484,-1.242704,1.061530,1.484948,-1.311919,-0.879427,0.565434,-0.703189,-0.149686,1.893964


## Attempts without Mongo

In [3]:
%%timeit
# NBVAL_IGNORE_OUTPUT
frame.to_parquet("test.parquet", engine="pyarrow")
pd.read_parquet("test.parquet", engine="pyarrow")

163 ms ± 5.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
# NBVAL_IGNORE_OUTPUT
frame.to_pickle("test.pickle")
pd.read_pickle("test.pickle")

39.2 ms ± 3.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
# NBVAL_IGNORE_OUTPUT
# note that here is no i/o to disk
x = Binary(pickle.dumps(frame))
pickle.loads(x)

4.14 ms ± 283 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Connecting to Mongo

In [6]:
MONGODB_SETTINGS = {'db': 'mongo', 'host': 'mongo'}

from mongoengine import *
connect(**MONGODB_SETTINGS)

MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## Using Parquet, Convert frame to parquet and back

In [7]:
from antarctic.pandas_fields import ParquetFrameField

class Maffay(Document):
    frame = ParquetFrameField()

In [8]:
%%timeit
# NBVAL_IGNORE_OUTPUT
m=Maffay(frame=frame).save()
m.frame

60.7 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Using Pickle, Convert using Pickle and back

In [10]:
from antarctic.pandas_fields import PicklePandasField

class Falco(Document):
    frame = PicklePandasField()